In [10]:
import json
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

In [11]:
def average_plot(X, y, sweep, interval, duration):
  sample_size = len(X[0])
  num_samples = len(X)
  arr0 = np.zeros(sample_size)
  arr1 = np.zeros(sample_size)
  count0 = np.zeros(sample_size)
  count1 = np.zeros(sample_size)
  for i in range(0,num_samples):
    for j in range(0,sample_size):
      cond_val = 6.0
      if(sweep == True):
        cond_val = 0
      if(y[i] == 0 and X[i][j] != cond_val):
        arr0[j] += X[i][j]
        count0[j] += 1
      if(y[i] == 1 and X[i][j] != cond_val):
        arr1[j] += X[i][j]
        count1[j] += 1
  for i in range(0,sample_size):
    if(count0[i] != 0):
      arr0[i] /= count0[i]
      if(sweep == True and arr0[i] != 0):
        arr0[i] = interval/arr0[i]
    if(count1[i] != 0):
      arr1[i] /= count1[i]
      if(sweep == True and arr1[i] != 0):
        arr1[i] = interval/arr1[i]
  ydata= []
  for i in range(1,101):
    ydata.append(i)
  ydata=np.array(ydata)
  plt.plot(arr0, '+--', label = "Non Target", linewidth=1, markersize=4,c="blue")
  plt.plot(arr1, 'o-', label = "Target", linewidth=1, markersize=4,c="orange",alpha=0.3)
  step=0.33
  plt.xticks(np.arange(0, sample_size+1, sample_size*step),[0,round(step*duration),round(step*2*duration),round(step*3*duration),round(step*4*duration)])
  plt.xlabel("Time of measurement (s)")
  plt.ylabel("Average buffer access time (ms)")
  plt.legend()
  plt.show()

In [12]:
def classify_LR(Xdata, ydata):
  skf = StratifiedShuffleSplit(n_splits=10,random_state=6)
  partial_accuracies=[]
  for train_index, test_index in skf.split(Xdata, ydata):
      X_train, X_test = Xdata[train_index], Xdata[test_index]
      y_train, y_test = ydata[train_index], ydata[test_index]
      mean=X_train.mean()
      std=X_train.std()
      X_train_norm=(X_train-mean+1e-10)/std
      X_test_norm=(X_test-mean+1e-10)/std
      model=LogisticRegression(max_iter=1000)
      model.fit(X_train_norm,y_train)
      prd=model.predict(X_test_norm)
      partial_accuracies.append(classification_report(y_test, prd, output_dict=True)['accuracy'])
  mean=round(np.mean(partial_accuracies)*100,2)
  deviation=round(np.std(partial_accuracies)*100,2)
  return ("Attack accuracy with logistic regression classifier: " + str(mean)+"%±"+str(deviation)+"%")

In [ ]:
f = open('template.json')
sweep = False; ## False for Windows Intel and macOS Intel, True for macOS M1
interval = 2; ## 2 for Windows Intel and macOS Inte, 10 for macOS M1
data = json.load(f,)
X=np.array(data['X'],dtype=np.float32)
y=np.array(data['y'])
average_plot(X, y, sweep, interval, duration=3)
print(classify_LR(X, y))
f.close()